In [1]:
from typing import List
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

from utils import *
from dispatch.utils import *

import dispatch
import importlib
# importlib.reload(dispatch)

from utils import *
# from dispatch.network import ScoringNet, PointEncoder, PositionalEncoder
from networks.encoders import PointEncoder
from dispatch.dispatch import Dispatch
from simulator.base_simulator import BaseSimulator

In [2]:
import gym

class Environment:
    def __init__(self) -> None:
        self.env = gym.make('CartPole-v0')
        self.state = None
        self.reset()

    def reset(self):
        self.state = self.env.reset()

    def step(self, action):
        '''
        returns: reward
        '''
        state, reward, done, info = self.env.step(action)
        self.state = state
        if done:
            self.reset()
            return 0
        return reward

    def get_state(self):
        '''
        returns the copy of state via inherence
        '''
        return self.state

In [3]:
class Policy:
    def __init__(self, model):
        self.model = model

    def estimate_value(self, state):
        inp = torch.tensor(state, dtype=torch.float32)
        return self.model(inp)[1]

    def act(self, state):
        inp = torch.tensor(state, dtype=torch.float32)
        actor, critic = self.model(inp)
    
        # probs = nn.functional.softmax(a, dim=-1).detach().numpy()
        # actions = np.array([np.random.choice(6, p=probs[i]) for i in range(len(inputs))])
        action_probs = nn.functional.softmax(actor, dim=-1)
        action = torch.distributions.Categorical(action_probs).sample()
        # logits = torch.gather(a, dim=1, index=torch.tensor(actions).unsqueeze(-1)).squeeze()

        log_softmaxed = nn.functional.log_softmax(actor, dim=-1)

        # print(log_softmaxed[action.unsqueeze(-1)])
        log_probs = torch.gather(log_softmaxed, dim=-1, index=action.unsqueeze(-1)).squeeze(-1)

        d = {}
        d['actions'] = action
        d['logits'] = actor
        d['log_probs'] = log_probs
        d['values'] = critic

        return d


In [4]:
class TestModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.body = nn.Sequential(
            nn.Linear(4, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU()
        )
        self.actor = nn.Linear(5, 2)
        self.critic = nn.Linear(5, 1)
    
    def forward(self, x):
        x = self.body(x)
        return self.actor(x), self.critic(x).squeeze(-1)

In [5]:
model = TestModel()
policy = Policy(model)
env = Environment()

d = policy.act(env.get_state())
env.step(d['actions'].item())

[2023-02-10 21:03:31,914] Making new env: CartPole-v0
c:\Users\Danill\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


1.0

In [6]:
from joblib import Parallel, delayed
from collections import defaultdict

class SessionCreater:
    def __init__(self, env, policy, n_steps=4, n_sessions=8, mode='parallel') -> None:
        self.env = env
        self.policy = policy
        self.n_steps = n_steps
        self.n_sessions = n_sessions
        self.mode = mode

    def generate_session(self):
        '''
        output trajectory: dict with keys:
        -states
        -actions
        -rewards

        -logits
        -log_probs
        
        -values
        -advantages
        '''
        trajectory = defaultdict(list)
        self.env.reset()

        for i in range(self.n_steps):
            trajectory['states'].append(self.env.get_state())
            act = self.policy.act(self.env.get_state())

            trajectory['actions'].append(act['actions'])
            trajectory['logits'].append(act['logits'])
            trajectory['log_probs'].append(act['log_probs'])
            trajectory['values'].append(act['values'])

            reward = self.env.step(trajectory["actions"][-1].item())
            trajectory['rewards'].append(reward)

        trajectory['last_state'] = {
            'state': self.env.get_state(),
            'value': self.policy.estimate_value(self.env.get_state())
        }

        return trajectory

    def parallel_generate_sessions(self):
        return Parallel(n_jobs=-1)(delayed(self.generate_session)() for i in range(self.n_sessions))
    
    def sequential_generate_sessions(self):
        return [self.generate_session() for i in range(self.n_sessions)]
    
    def __call__(self):
        if self.mode == 'parallel':
            return self.parallel_generate_sessions()
        return self.sequential_generate_sessions()

In [7]:
def compute_estimated_cumulative_rewards(rewards, last_state_value, gamma=0.99):
    '''
    input: 
    -rewards: List[float] 
    -last_state_q_value: float 
    output: 
    -cum_rewards: List[float]
    '''
    reverse_rewards = [last_state_q_value]
    n_steps = len(rewards)
    for i in range(n_steps, 0, -1):
        reverse_rewards.append(reverse_rewards[-1] * gamma + rewards[i - 1])
    return reverse_rewards[1:][::-1]

In [8]:
class ComputeValueTargets:
    def __init__(self, gamma=0.99):
        self.gamma = gamma

    def compute_cumutitive_rewards(self, rewards, last_state_value):
        reverse_rewards = [last_state_value]
        n_steps = len(rewards)
        for i in range(n_steps, 0, -1):
            reverse_rewards.append(reverse_rewards[-1] * self.gamma + rewards[i - 1])
        return reverse_rewards[1:][::-1]
    
    def __call__(self, session):
        for trajectory in session:
            rewards = trajectory['rewards']
            last_state_value = trajectory['last_state']['value']
            trajectory['targets'] = self.compute_cumutitive_rewards(rewards, last_state_value)

In [10]:
cvt = ComputeValueTargets()
s = SessionCreater(env, policy, mode='def')
trs = s()
cvt(trs)
trs[0]

defaultdict(list,
            {'states': [array([-0.03555699, -0.02499039,  0.00473004,  0.01856952]),
              array([-0.03605679, -0.22017985,  0.00510143,  0.31274107]),
              array([-0.04046039, -0.02513094,  0.01135625,  0.02167133]),
              array([-0.04096301,  0.16982632,  0.01178968, -0.26740704])],
             'actions': [tensor(0), tensor(1), tensor(1), tensor(0)],
             'logits': [tensor([-4.1165e-01, -1.1660e-04], grad_fn=<AddBackward0>),
              tensor([-0.4294, -0.0023], grad_fn=<AddBackward0>),
              tensor([-0.4124, -0.0007], grad_fn=<AddBackward0>),
              tensor([-0.3986, -0.0016], grad_fn=<AddBackward0>)],
             'log_probs': [tensor(-0.9199, grad_fn=<SqueezeBackward1>),
              tensor(-0.5022, grad_fn=<SqueezeBackward1>),
              tensor(-0.5083, grad_fn=<SqueezeBackward1>),
              tensor(-0.9113, grad_fn=<SqueezeBackward1>)],
             'values': [tensor(0.2142, grad_fn=<SqueezeBackward1>),


In [123]:
def merge_time_batch(sessions):
    keys = [k for k in sessions[0].keys() if k != 'last_state']
    result = {}
    for key in keys:
        result[key] = torch.hstack([s[key] for s in sessions])

    return result

In [125]:
merge_time_batch(trs)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [8]:
def convert_sessions_to_tensors(trajectory):
    # trajectory['targets'] = compute_estimated_cumulative_rewards(trajectory['rewards'], trajectory['last_state']['q_value'])
    for k, v in trajectory.items():
        pass

def compute_losses():
    '''
    input:
    -
    output: 
    -policy_loss 
    -value_loss (V = G) 
    -entropy_loss
    '''
    pass